# Concurrency with `asyncio`

`asyncio`는 Python 3.4 부터 공개가 되고 그 이후에도 계속 문법이 바뀌었습니다. 책은 3.4를 기준으로 작성이 되었기에, 이제는 거의 쓰이지 않는 옛날 문법으로 작성이 되어있습니다. 읽기에도 힘들고 이해도 잘 안되지요. 그래서, 인터넷에서 많이 읽어보고 좋은 링크들을 정리해놨습니다.

## Some Useful Links

| 링크 | 설명 |
|--|--|
| [goroutine vs python coroutine](https://www.quora.com/What-is-the-difference-between-Python-asyncio-and-Golang-Go-routines) | goroutine vs coroutine에 대해 quora 링크가 가장 정리가 잘 되어있는 것 같습니다. 아주 간단히 말하자면:<br>- coroutine은 IO-bound 연산에 좋습니다. GIL때문에 CPU-bound 연산은 좋지 않고, built-in IO 연산은 C로 작성되어 빠르죠.<br>- goroutine은 CPU-bound 연산에 좋습니다. 모든 프로세서를 다 쓰기 때문이죠.|
| [Removing GIL from python](http://python-notes.curiousefficiency.org/en/latest/python3/multicore_python.html) | 왜 GIL을 없애지 못하는걸까요? 문서의 "Why isn’t “just remove the GIL” the obvious answer?" 부분을 보세요.<br>(1) GIL을 가정하고 내부를 짜서 내부가 thread-safe 하지 못함.<br>(2) C로 만든 built-in들은 GIL을 가정해야 서로 서로 concurrent 하게 잘 돌아감.  |
| [John Reese - Thinking Outside the GIL with AsyncIO and Multiprocessing - PyCon 2018](https://www.youtube.com/watch?v=0kXaLh8Fz3k) | 페이스북에서 사용한 asyncio예제입니다. 모니터링을 하는 워크로드가 있는데 이런특징이 있었다네요. <br><br>- 데이터를 수십만 수백만 데이터포인트에서 긁어옴. <br>- 간단히 anomaly detection 을 함 (아주 간단해서 CPU-bound 아님)<br>- abnormal 한 데이터들은 DB에 저장 <br><br>결론적으로 IO-bound워크로드라는거네요. asyncio가 잘 돌아갈 것 같은 느낌이죠? 너무 간단한 것이라서 파이썬으로 작성을 했다는데요. 당연히 시간이 지나서 더 많은 서버에서 작업을 해야하니 잘 안돌아갔죠. <br><br> 여러 실험을 했습니다: threading, asyncio, multiprocessing, multiprocessing + asyncio + tuning + workload를 map/reduce 스타일로 바꿈<br> 당연히 가장 마지막 버전이 잘 돌았다고 해요. 여러 프로세스를 돌리면서 GIL 의 불합리함도 없애고, asyncio도 잘 사용하구요. 그 과정을 열심히 보여줍니다. <br><br>[Github으로 오픈소스](https://github.com/jreese/aiomultiprocess)도 했습니다. 나중에 써봐야겠습니다.|
| [Yury Selivanov - Asyncio in Python 3 7 and 3 8](https://youtu.be/ReXxO_azV-w) | 이분이 파이썬 asyncio 핵심 개발자입니다. 매년 컨퍼런스를 다니며 버전이 오를때마다 바뀌는 asyncio 를 설명하네요.|
| [Python tricks: Demystifying async, await, and asyncio](https://www.youtube.com/watch?v=tSLDcRkgTsY) | 15분짜리 영상인데 입문으로 가장 좋은 것 같습니다. 여기의 예제를 이 노트북에서 사용하겠습니다. |

## History

[Yury Selivanov - Asyncio in Python 3 7 and 3 8](https://youtu.be/ReXxO_azV-w) 에서 정리해옵니다.

| 버전 | 주요 변화 |
|--|--|
| 3.4 | coutines via `yield from`|
| 3.5 | `async` and `await` |
| 3.6 | `get_event_loop()` |
| 3.7 | `asyncio.run()` |
| 3.8 | |